In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Any results you write to the current directory are saved as output.
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/recommendationsv4/train.csv')
test = pd.read_csv('/kaggle/input/recommendationsv4/test.csv')
submission = pd.read_csv('/kaggle/input/recommendationsv4/sample_submission.csv')

# EDA

Датасет

У вас будет история оценок пользователя вместе с его обзором. Вы можете использовать текст рецензии в качестве дополнительной информации. Все оценки пользователей нормированы для бинарной классификации, если человек поставил оценку продукту больше 3 (не включительно), то мы считаем, что продукт ему понравился, если меньше 4, то продукт не понравился.

Составлю список характеристик признаков train:

* rating - понравилась или не понравилась книга (1 - понравилась, 0 - не понравилась). 

test.csv - набор данных, для которого вы должны сделать предсказания. У каждого наобора userid, itemid есть свой id, для которого вы должны сделать предсказание:

- overall - рейтинг, который поставил пользователь
- verified - был ли отзыв верифицирован
- reviewTime - когда был отзыв написан
- reviewerName - имя пользователя
- reviewText - текст отзыва
- summary - сжатый отзыв
- vote - количество голосований за отзыв
- style - метаданные
- image - изображение продукта
- userid - id пользователя
- itemid - id товара
- id - id для предсказания

Посмотрим на train

In [ ]:
train.sample(15)

In [ ]:
test.sample(5)

ASIN является уникальным для листинга продукта, вы его получаете от Амазон вместо UPC кода. Назову это условно "товар".

# Ок, выделю товары в отдельный список и попробую просмотреть на рейтинги, лайки и тд.

Создам базу товаров (из теста и трейна) - для этого возьму itemid и asin, а потом сгруппирую по одному из признаков (по идее не важно по какому, это одно и то же, но проверим).

Для будущих интересностей еще сделаю еще несколько колонок в списке товаров - положительные и негативные оценки. Буду ориентироваться  на 1 или 0 от пользователя. Если 0 - негативная оценка. Если 1 - положительная.

In [ ]:
pd.options.mode.chained_assignment = None 

# возьму нужные колонки из трейна и создам новый датасет с товарами
items_train_list = train[['asin', 'itemid', 'rating']]

items_train_list['likes_up'] = np.where(items_train_list['rating'] == 1, 1, 0)
items_train_list['likes_down'] = np.where(items_train_list['rating'] == 0, 1, 0)

# сгруппирую товары по asin и посмотрю, сколько itemid входит
items_train_list_grouped_by_likes = items_train_list.groupby(['asin']) \
    .agg({'itemid': 'count', 'likes_up': 'sum', 'likes_down': 'sum'}) \
    .sort_values(by=['asin'], ascending=False)\
    .reset_index()\
    .rename(columns={'itemid': 'all_likes'})

items_train_list_grouped_by_likes

Добавлю также средний рейтинг товара

In [ ]:
items_train_list_with_overall = train[['asin', 'itemid', 'overall']]

# сгруппирую товары по asin и посмотрю средний рейтинг
items_train_list_grouped_by_overall_mean = items_train_list_with_overall.groupby(['asin']) \
    .agg({'overall': 'mean'}) \
    .sort_values(by=['asin'], ascending=False) \
    .reset_index() \
    .rename(columns={'overall': 'item_rating_mean'})

items_train_list_grouped_by_overall_mean

Склею два датасета товаров

In [ ]:
items_train_list = items_train_list_grouped_by_overall_mean.merge(items_train_list_grouped_by_likes, left_on='asin', right_on='asin')
items_train_list = items_train_list.sort_values(by='all_likes', ascending=False)
items_train_list

Посмотрим на распределение количества лайков и книг

In [ ]:
items_train_list['all_likes'].hist(bins='doane')

In [ ]:
items_train_list[items_train_list['all_likes'] > 100]

Что мы видим выше - что только около 1200 товаров из 41000 имеют количество лайков больше 100. Остальные товары можно отнести к LongTail... Что это значит для нас на практике?

Это значит, что мы можем применить это для "холодного старта".

Существует очень маленькое количество айтемов, о которых знают все. Нет никакого смысла их рекомендовать, потому что пользователь, скорее всего, либо уже их видел и просто не поставил оценку, либо и так о них знает и собирается посмотреть, либо твердо решил не смотреть вовсе. Я не раз смотрел трейлер «Списка Шиндлера», но посмотреть так и не собрался. С другой стороны, популярность очень быстро спадает, и подавляющее количество айтемов практически никто не видел. Делать рекомендации из этой части полезней: там есть интересный контент, который пользователь вряд ли сможет найти сам.

А еще для "холодного старта" мы можем на старте как-то опросить пользователя на предмет того, какие ему товары нравятся (выбор из списка), как делает тот же Pinterest (или другие, где со старта мне предлагают оценить мои интересы). Или если мы говорим про товары, то можем ему со старта показать 5 самых популярных наших товаров (которые имеют больше всего голосов), которые он может отметить голосами - нравится или не нравится. И мы уже будем знать, что ему примерно нравится.

Как не сортировать по среднему рейтингу и учитывать количество оценок? Посчитать доверительный интервал: «Исходя из имеющихся оценок, с вероятностью в 95 % истинная доля положительных оценок как минимум какая?». Ответ на этот вопрос дал Эдвин Уилсон в 1927 году: http://www.evanmiller.org/how-not-to-sort-by-average-rating.html

In [ ]:
from math import sqrt

def confidence(ups, downs):
    n = ups + downs

    if n == 0:
        return 0

    z = 1.96 #1.44 = 85%, 1.96 = 95%
    phat = float(ups) / n
    return ((phat + z*z/(2*n) - z * sqrt((phat*(1-phat)+z*z/(4*n))/n))/(1+z*z/n))

items_train_list['item_score'] = items_train_list.apply(lambda row : confidence(row['likes_up'], row['likes_down']), axis = 1)

items_train_list

Посмотрим-ка на распределение score

In [ ]:
items_train_list['item_score'].hist(bins='doane')

Довольно интересное распределение. С натяжкой вроде бы можно отнести к нормальному распределению.

Пока что провел некоторые операции с базой товаров - это больше интересно для холодного старта, ориентируясь на рейтинги товара. Пока что я вижу так. А теперь продолжим работать с пользователями, лайками из датасета..

# Работа с train

### Image

Сразу видно, что в колонке image очень много пропусков. Побороть это не представляется возможным. Но так как это только 1% от общей выборки, то можно смело удалить признак.

In [ ]:
train = train.drop(columns='image')

### unixReviewTime, reviewTime

Проверю, что выдает unixReviewTime при переводе его в нормальный формат времени.

In [ ]:
train['reviewTime_new'] = pd.to_datetime(train.unixReviewTime, unit='s')

Видим, что новая колонка полностью дублирует колонку reviewTime, но уже в нормальном формате. Следовательно, теперь reviewTime можно смело удалить.

Удалять буду так, как того требует команда pandas путем замены текущего датафрейма, без inplace=True

In [ ]:
train = train.drop(columns=['reviewTime'])

И сразу переименую колонку reviewTime_new в нормальное название

In [ ]:
train = train.rename(columns={'reviewTime_new': 'review_time', 'unixReviewTime': 'unix_review_time'})

In [ ]:
train

### Rating

Выяснили, что rating - это нравится книга или нет. Переведу это в int.

In [ ]:
train['rating'] = train['rating'].astype(int)

Посмотрю пристальнее на признаки. Создам функцию просмотра данных:

In [ ]:
def col_info(col, bins):
    print('Количество пропусков: {}'.format(col.isnull().sum()))
    print('{},'.format(col.describe()))
    print('Распределение:\n{},'.format(col.value_counts()))
    col.hist(bins=bins)

### Overall

In [ ]:
col_info(train.overall, 'doane')

Переведу в int признак overall.

In [ ]:
train['overall'] = train['overall'].astype(int)

### Verified

In [ ]:
train.verified.nunique()

In [ ]:
train.verified.unique()

Что это зна признак пока не понятно. Пропусков нет. Варианты - True или False. Есть предположение, что речь идет о том, что отзыв был заапрувлен (опубликован на сайте) или нет.

Надо ли ориентироваться на этот признак? В сущности, человек так или иначе оставил свой отзыв и это его мнение о книге. Значит, можно удалить и этот признак.

In [ ]:
train = train.drop(columns='verified')

### Vote

In [ ]:
col_info(train.vote, 'doane')

Очень много пропусков... Сделать с этим ничего нельзя - удалю признак.

In [ ]:
train = train.drop(columns='vote')

### reviewerName	

Думаю, что нет смысла ориентироваться на этот признак - он ничего нам не дает.

In [ ]:
train = train.drop(columns='reviewerName')

In [ ]:
train.sample(5)

Попробую добавить новые признаки на основании даты отзыва - год, месяц, день, день недели.

In [ ]:
train['review_year'] = pd.DatetimeIndex(train['review_time']).year
train['review_month'] = pd.DatetimeIndex(train['review_time']).month
train['review_day'] = pd.DatetimeIndex(train['review_time']).day
train['review_dayofweek'] = pd.DatetimeIndex(train['review_time']).dayofweek.astype(object)

# перенесу этот признак наперед, потому что он у меня объект и мне будет удобнее провести OneEncoding для CatBoost
cols = ['review_dayofweek']  + [col for col in train if col != 'review_dayofweek']
train = train[cols]

train = train.drop(columns='review_time')
train

Так как в тестовой выборке нет колонки overall, удалю ее и в трейне

In [ ]:
train = train.drop(columns='overall')

### style

Тоже самое с колонкой style

In [ ]:
train = train.drop(columns='style')

### reviewText, summary

Так как в тесте нету reviewText, summary, то удалю их - все равно не смогу применить эти признаки.

In [ ]:
train = train.drop(columns=['summary', 'reviewText'])

In [ ]:
train

# Работа с test

In [ ]:
test = pd.read_csv('/kaggle/input/recommendationsv4/test.csv')

In [ ]:
test = test.drop(columns='image')
test['reviewTime_new'] = pd.to_datetime(test.unixReviewTime, unit='s')
test = test.drop(columns=['reviewTime'])
test = test.rename(columns={'reviewTime_new': 'review_time', 'unixReviewTime': 'unix_review_time'})
test = test.drop(columns=['vote', 'reviewerName'])
test = test.drop(columns='verified')
test['review_year'] = pd.DatetimeIndex(test['review_time']).year
test['review_month'] = pd.DatetimeIndex(test['review_time']).month
test['review_day'] = pd.DatetimeIndex(test['review_time']).day
test['review_dayofweek'] = pd.DatetimeIndex(test['review_time']).dayofweek.astype(object)

# перенесу этот признак наперед, потому что он у меня объект и мне будет удобнее провести OneEncoding для CatBoost
cols = ['review_dayofweek']  + [col for col in test if col != 'review_dayofweek']
test = test[cols]

test = test.drop(columns='review_time')
test = test.drop(columns='Id')
test = test.drop(columns='style')


In [ ]:
test.info()

# Fit/Predict

Ну что же, первичные преобразование сделал. Попробую теперь получить какое-то предсказание с помощью LightFM - это будет мой бейслайн.

In [ ]:
import scipy.sparse as sparse

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
import sklearn
from sklearn.model_selection import train_test_split

import scipy.sparse as sparse

In [ ]:
train_data, test_data = train_test_split(train,random_state=32, shuffle=True)

In [ ]:
ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))

In [ ]:
NUM_THREADS = 4 #число потоков
NUM_COMPONENTS = 70 #число параметров вектора 
NUM_EPOCHS = 20 #число эпох обучения

model = LightFM(learning_rate=0.1, loss='logistic',
                learning_schedule='adagrad',
                no_components=NUM_COMPONENTS)
model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

In [ ]:
preds_lightfm = model.predict(test_data.userid.values,
                      test_data.itemid.values)


In [ ]:
preds_lightfm

In [ ]:
preds_lightfm.min(), preds_lightfm.max()

In [ ]:
normalized_preds_lightfm = (preds_lightfm - preds_lightfm.min())/(preds_lightfm - preds_lightfm.min()).max()

In [ ]:
normalized_preds_lightfm

In [ ]:
sklearn.metrics.roc_auc_score(test_data.rating,preds_lightfm)


In [ ]:
normalized_preds_lightfm.min(), normalized_preds_lightfm.max()

In [ ]:
preds_lightfm = model.predict(test.userid.values,
                      test.itemid.values)


In [ ]:
preds_lightfm.min(), preds_lightfm.max()

In [ ]:
test_normalized_preds_lightfm = (preds_lightfm - preds_lightfm.min())/(preds_lightfm - preds_lightfm.min()).max()

In [ ]:
normalized_preds_lightfm.min(), normalized_preds_lightfm.max()

# Predict Upper

Попробую теперь улучшить модель. Для этого буду вводить новые признаки в датафреймы и с помощью различных ухищрений и библиотеки CatBoost попробую улучшить прогноз для колонки rating.

Добавлю к таблицам трейн и тест данные из таблицы товаров

In [ ]:
test_full = test.copy()
train_full = train.copy()

CatBoost выдает значительно хуже результат, если в данные не добавить значения о книгах.

In [ ]:
train_full[['unix_review_time', 'userid', 'itemid', 'rating', 'review_year', 'review_month', 'review_day']] = train_full[['unix_review_time', 'userid', 'itemid', 'rating', 'review_year', 'review_month', 'review_day']].astype('int32')
test_full[['unix_review_time', 'userid', 'itemid', 'review_year', 'review_month', 'review_day']] = test_full[['unix_review_time', 'userid', 'itemid', 'review_year', 'review_month', 'review_day']].astype('int32')

In [ ]:
train_full

In [ ]:
test_full

In [ ]:
train_full = train.merge(items_train_list, how='left', left_on='asin', right_on='asin')
test_full = test.merge(items_train_list, how='left', left_on='asin', right_on='asin')

После того как добавил в тестовый датасет данные из колонки товары, выяснилось, что в тесте нет данных для 60 товаров. На фоне общего количества это мизер. Поэтому просто добавлю сюда срение значения по выборе и все.

In [ ]:
test_full['item_rating_mean'].fillna((test_full['item_rating_mean'].mean()), inplace=True)
test_full['all_likes'].fillna((test_full['all_likes'].mean()), inplace=True)
test_full['likes_up'].fillna((test_full['likes_up'].mean()), inplace=True)
test_full['likes_down'].fillna((test_full['likes_down'].mean()), inplace=True)
test_full['item_score'].fillna((test_full['item_score'].mean()), inplace=True)

Ранее с помощью K-FOLD техними получил с помощью LightFM предсказания для всех трейновых записей. Чтобы не засорять страницу с кодом, вынес это в отдельный файл. Теперь подключил его и далее добавлю данные в датасеты для второй модели.

In [ ]:
train_target_predict_from_lightfm = pd.read_csv('../input/train-target-predict/train_target_predict_from_lightfm.csv')

Теперь добавлю предсказания, полученные от LightFM для каждого наблюдения. И буду на них тренировать вторую модель.

In [ ]:
train_full['lightfm_rating'] = train_target_predict_from_lightfm

test_full['lightfm_rating'] = pd.DataFrame(test_normalized_preds_lightfm)

In [ ]:
test_full

Нам по ходу не нужен признак itemid, так как оно коррелирует с asin. А зачем userid? Тоже вроде не нужен

In [ ]:
test_full = test_full.drop(columns=['itemid'])
train_full = train_full.drop(columns=['itemid'])

In [ ]:
test_full = test_full.drop(columns=['asin'])
train_full = train_full.drop(columns=['asin'])

In [ ]:
test_full = test_full.drop(columns=['itemid', 'asin', 'all_likes', 'likes_up', 'likes_down'])
train_full = train_full.drop(columns=['itemid', 'asin', 'all_likes', 'likes_up', 'likes_down'])

Попробую сгенерировать признаки на основании временного окна

In [ ]:
train_full['lightfm_rolling_mean'] = train_full['lightfm_rating'].rolling(window=7).mean()
train_full['lightfm_rolling_mean'] = train_full['lightfm_rolling_mean'].fillna(0)

test_full['lightfm_rolling_mean'] = test_full['lightfm_rating'].rolling(window=7).mean()
test_full['lightfm_rolling_mean'] = test_full['lightfm_rolling_mean'].fillna(0)

In [ ]:
train_full['lightfm_expanding_mean'] = train_full['lightfm_rating'].expanding(2).mean()
train_full['lightfm_expanding_mean'] = train_full['lightfm_expanding_mean'].fillna(0)

test_full['lightfm_expanding_mean'] = test_full['lightfm_rating'].expanding(2).mean()
test_full['lightfm_expanding_mean'] = test_full['lightfm_expanding_mean'].fillna(0)

Этот код выбирает числовые признаки, вычисляет их логарифмы, выбирает два категориальных признака, применяет к ним one-hot кодирование и объединяет оба множества в одно, а таже суммирует значения признаков между собой.

In [ ]:
import itertools

def itr_combinations_new_features(data):
    
    tmp_df = data.copy()
    
    old_columns = [item for item in tmp_df.columns.tolist() if item != 'rating' and item != 'review_dayofweek' and item != 'userid']
    
    tmp_df_sum_2 = [(pd.Series(tmp_df.loc[:,list(i)].sum(axis=1),\
            name='_sum2_'.join(tmp_df.loc[:,list(i)].columns))) for i in list(itertools.combinations(old_columns,2))]
    
    tmp_df_sum_3 = [(pd.Series(tmp_df.loc[:,list(i)].sum(axis=1),\
            name='_sum3_'.join(tmp_df.loc[:,list(i)].columns))) for i in list(itertools.combinations(old_columns,3))]
    
    tmp_df_multiplication = [(pd.Series(tmp_df.loc[:,i[0]]*tmp_df.loc[:,i[1]],\
            name='_mult_'.join(tmp_df.loc[:,list(i)].columns))) for i in list(itertools.combinations(old_columns,2))]
    
   
    df_sum_2 = pd.DataFrame(tmp_df_sum_2).T  
    
    df_sum_3 = pd.DataFrame(tmp_df_sum_3).T  
    
    df_multiplication = pd.DataFrame(tmp_df_multiplication).T  
   
    data = pd.concat([data, df_sum_2, df_sum_3, df_multiplication], axis=1, sort=False)         
 
    
    return data

In [ ]:
train_full = itr_combinations_new_features(train_full)
train_full = train_full.fillna(0)

In [ ]:
train_full

In [ ]:
test_full = itr_combinations_new_features(test_full)
test_full = test_full.fillna(0)

In [ ]:
test_full.info()

In [ ]:
def code_mean(data, cat_feature, real_feature):
    """
    Returns a dictionary where keys are unique categories of the cat_feature,
    and values are means over real_feature
    """
    return dict(data.groupby(cat_feature)[real_feature].mean())

In [ ]:
average_dayofweek = code_mean(train_full, 'review_day', "rating")
plt.figure(figsize=(7, 5))
plt.title("dayofweek averages")
pd.DataFrame.from_dict(average_dayofweek, orient='index')[0].plot()
plt.grid(True);

In [ ]:
test_full

In [ ]:
train_full["lightfm_rating_by_day_average"] = list(map(code_mean(train_full, 'review_day', "lightfm_rating").get, train_full.review_day))
test_full["lightfm_rating_by_day_average"] = list(map(code_mean(test_full, 'review_day', "lightfm_rating").get, test_full.review_day))

train_full["review_by_month_average"] = list(map(code_mean(train_full, 'review_month', "lightfm_rating").get, train_full.review_month))
test_full["review_by_month_average"] = list(map(code_mean(test_full, 'review_month', "lightfm_rating").get, test_full.review_month))

train_full["lightfm_rating_by_dayofweek_average"] = list(map(code_mean(train_full, 'review_dayofweek', "lightfm_rating").get, train_full.review_dayofweek))
test_full["lightfm_rating_by_dayofweek_average"] = list(map(code_mean(test_full, 'review_dayofweek', "lightfm_rating").get, test_full.review_dayofweek))

Подчищу уже ненужные признаки

In [ ]:
test_full = test_full.drop(columns=['review_year', 'review_month', 'review_day', 'unix_review_time'])
train_full = train_full.drop(columns=['review_year', 'review_month', 'review_day', 'unix_review_time'])

In [ ]:
train_full

In [ ]:
test_full.info()

In [ ]:
train_full[['lightfm_rating', 'lightfm_rolling_mean', 'lightfm_expanding_mean', 'lightfm_rating_by_day_average', 'review_by_month_average', 'lightfm_rating_by_dayofweek_average']] = train_full[['lightfm_rating', 'lightfm_rolling_mean', 'lightfm_expanding_mean', 'lightfm_rating_by_day_average', 'review_by_month_average', 'lightfm_rating_by_dayofweek_average']].astype('float32')
test_full[['lightfm_rating', 'lightfm_rolling_mean', 'lightfm_expanding_mean', 'lightfm_rating_by_day_average', 'review_by_month_average', 'lightfm_rating_by_dayofweek_average']] = test_full[['lightfm_rating', 'lightfm_rolling_mean', 'lightfm_expanding_mean', 'lightfm_rating_by_day_average', 'review_by_month_average', 'lightfm_rating_by_dayofweek_average']].astype('float32')

In [ ]:
col_info(train_full.lightfm_rating, 'doane')

In [ ]:
col_info(train_full.item_score, 'doane')

Плохо распределен признак item_score. Логарифмировать не получится. Надо подумать, что можно сделать.

In [ ]:
train_full

In [ ]:
test_full

Применю теперь модель CatBoost Regressor

Но для начала удалю из трейна колонку rating, так как по сути теперь учить модель я буду на тех результатах, которые предсказал LightFM. Ну и в тесте удалю этот признак - его и будем учить.

In [ ]:
test_full = test_full.drop(columns=['lightfm_rating', 'userid'])
train_full = train_full.drop(columns=['rating', 'userid'])

In [ ]:
from catboost import CatBoostRegressor
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.model_selection import train_test_split

X = train_full.drop(['lightfm_rating'], axis=1,)
y = train_full.lightfm_rating.values

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)

X_test = test_full.drop([], axis=1,)

Байесовская оптимизация параметров CatBoost с Hyperopt

In [ ]:
import hyperopt

def hyperopt_objective(params):
    model = CatBoostRegressor(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=100,
        eval_metric='AUC',
        random_seed=42,
        verbose=False,
        loss_function='RMSE',
    )
    
    cv_data = cv(
        Pool(X_train, y_train),
        model.get_params()
    )
    best_accuracy = np.max(cv_data['test-AUC-mean'])
    
    return 1 - best_accuracy # as hyperopt minimises

In [ ]:
from numpy.random import RandomState

params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 0.01, 0.5),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=10,
    trials=trials,
    rstate=RandomState(123)
)

print(best)

In [ ]:
# CatBoost Regression

params = {
    'iterations': 400,
    'learning_rate': 0.2,
    'l2_leaf_reg': 3,
    'eval_metric': 'AUC',
    'loss_function': 'RMSE',
    'use_best_model': True,
    'random_seed': 42,
    'logging_level': 'Silent'
}


model = CatBoostRegressor(**params)

categorical_features_names = [0]

# Fit model
model.fit(
    X_train, y_train,
    cat_features=categorical_features_names,
    eval_set=(X_validation, y_validation),
#     logging_level='Verbose',  # you can uncomment this for text output
    plot=True
);

Посмотрим, какие признаки CatBoost считает важными.

In [ ]:
from catboost import Pool, CatBoostRegressor, cv

feature_score = pd.DataFrame(list(zip(X.dtypes.index, model.get_feature_importance(Pool(X_train, label=y_train, cat_features=categorical_features_names)))),
                columns=['Feature','Score'])

feature_score = feature_score.sort_values(by='Score', ascending=False, inplace=False, kind='quicksort', na_position='last')

feature_score

Удалю ненужные по-моему мнению признаки.

In [ ]:
def del_bag_feat(data):

    feature_score_upd = feature_score[(feature_score['Score']<0.3)]
    feature_score_upd = feature_score_upd['Feature'].tolist()
    
    print(feature_score_upd)
    
    indexes_to_keep = set(data.columns) - set(feature_score_upd)    
   
    data = data[indexes_to_keep]
    

    return data

In [ ]:
train_full2 = del_bag_feat(train_full)

In [ ]:
train_full2

In [ ]:
test_full2 = del_bag_feat(test_full)

In [ ]:
test_full2

А теперь удалю еще признаки, которые коррелируют друг с другом.

In [ ]:
def drop_corr_features(data):
    # Create correlation matrix
    corr_matrix = data.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    # Find features with correlation greater than 0.95
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

    print(to_drop)
    
    data.drop(to_drop, axis=1, inplace=True)

    # Drop features 
    return data

In [ ]:
train_full = drop_corr_features(train_full)

In [ ]:
test_full = drop_corr_features(test_full)

In [ ]:
train_full

In [ ]:
test_full

In [ ]:
# перенесу этот признак наперед, потому что он у меня объект и мне будет удобнее провести OneEncoding для CatBoost
cols = ['review_dayofweek']  + [col for col in train_full if col != 'review_dayofweek']
train_full = train_full[cols]

In [ ]:
# перенесу этот признак наперед, потому что он у меня объект и мне будет удобнее провести OneEncoding для CatBoost
cols = ['review_dayofweek']  + [col for col in test_full if col != 'review_dayofweek']
test_full = test_full[cols]

Запущу теперь заново обучение с уже почищенными данными.

In [ ]:
from sklearn.model_selection import train_test_split

X = train_full2.drop(['lightfm_rating'], axis=1,)
y = train_full2.lightfm_rating.values

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)

X_test = test_full2.drop([], axis=1,)

In [ ]:
# CatBoost Regression

params = {
    'iterations': 450,
    'learning_rate': 0.1,
    'l2_leaf_reg': 2,
    'eval_metric': 'AUC',
    'loss_function': 'RMSE',
    'use_best_model': True,
    'random_seed': 42,
    'logging_level': 'Silent'
}


model = CatBoostRegressor(**params)

# categorical_features_names = [0]

# Fit model
model.fit(
    X_train, y_train,
#     cat_features=categorical_features_names,
    eval_set=(X_validation, y_validation),
#     logging_level='Verbose',  # you can uncomment this for text output
    plot=True
);

In [ ]:
from catboost import Pool, CatBoostRegressor, cv

feature_score = pd.DataFrame(list(zip(X.dtypes.index, model.get_feature_importance(Pool(X_train, label=y_train)))),
                columns=['Feature','Score'])

feature_score = feature_score.sort_values(by='Score', ascending=False, inplace=False, kind='quicksort', na_position='last')

feature_score

In [ ]:
def del_bag_feat_2(data):

    feature_score_upd = feature_score[(feature_score['Score']<2)]
    feature_score_upd = feature_score_upd['Feature'].tolist()
    
    print(feature_score_upd)
    
    indexes_to_keep = set(data.columns) - set(feature_score_upd)    
   
    data = data[indexes_to_keep]
    

    return data

In [ ]:
train_full2 = del_bag_feat_2(train_full2)
test_full2 = del_bag_feat_2(test_full2)

In [ ]:
train_full2.info()

In [ ]:
# Get predictions
predictions_catboost = model.predict(X_test)
print(predictions_catboost[:10])

In [ ]:
plt.rcParams["figure.figsize"] = (12,7)
ax = feature_score.plot('Feature', 'Score', kind='bar', color='c')
ax.set_title("Catboost Feature Importance Ranking", fontsize = 14)
ax.set_xlabel('')

rects = ax.patches

labels = feature_score['Score'].round(2)

for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 0.35, label, ha='center', va='bottom')

plt.show()

In [ ]:
model = CatBoostRegressor(
    l2_leaf_reg=int(best['l2_leaf_reg']),
    learning_rate=best['learning_rate'],
    iterations=750,
    eval_metric='AUC',
    random_seed=42,
    verbose=True,
    loss_function='RMSE',
)


In [ ]:
cv_data = cv(Pool(X_train, y_train), model.get_params())

In [ ]:
cv_data

In [ ]:
model.fit(X_train, y_train, plot=True)

In [ ]:
# Get predictions
predictions_catboost = model.predict(X_test)
print(predictions_catboost[:10])

Теперь хочу просто перебрать много вариантов предсказаний катбуста с разными параметрами и просто усредить и посмотреть, что получится.

In [ ]:
RANDOM_STATE = 900

CATBOOST_PARAMS_GENERATOR_VER = 11

catbr_params = {
    'random_seed': RANDOM_STATE, 
    'learning_rate': 0.1, #0.2-оптимальный параметр
    'iterations': 750, #750-оптимальный параметр
    'depth': 9, # 9-оптимальный параметр
    'l2_leaf_reg': 2, # 2-оптимальный параметр
    'eval_metric': 'AUC',
    'subsample' : 0.75,
    'random_strength': 0.06, 
    'od_type': "Iter", #детектор переобучения catboost
    'od_wait': 1500,  # 1500-оптимальный параметр 
    'loss_function': 'RMSE',
    'verbose': False, 
}

In [ ]:
import random

col_mean = []

result_pred_df = pd.DataFrame()

for i in range(30):
    # Params rendomizer
    catbr_params['subsample'] = random.choice([0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95])
    catbr_params['learning_rate'] = random.choice([0.3, 0.1, 0.2, 0.4])
    catbr_params['depth'] = random.choice([2, 4, 6, 7, 8, 9])
    catbr_params['l2_leaf_reg'] = random.choice([0.85, 0.9, 1, 2, 3, 4, 2.5, 3.5, 4.5])
    catbr_params['random_seed'] = random.randint(10,10000)
    predict_model_ = 'result' + str(CATBOOST_PARAMS_GENERATOR_VER) + str(i) + "_subs_" + str(catbr_params['subsample'])+  "_lr_" + str(catbr_params['learning_rate'])+ "_l2_" + str(catbr_params['l2_leaf_reg'])+\
            "_rs_" + str(catbr_params['random_seed'])
      
    # Model
    model = CatBoostRegressor(**catbr_params)
    
    # Model fit
    model.fit(
    X_train, y_train,
    eval_set=(X_validation, y_validation),
#     cat_features=categorical_features_names,
    plot=False
    );
    
    print(i)
    
    # Get predictions
    predict_model_ = model.predict(X_test)
    result_pred_df[f'result_{i}'] = predict_model_
    print(predict_model_[:10])    

    col_mean.extend([predict_model_])
    
print(col_mean)

In [ ]:
col_list = result_pred_df.columns.tolist()
result_pred_df['y_pred_v0'] = result_pred_df[col_list].mean(axis = 1)

In [ ]:
result_pred_df

In [ ]:
predictions_catboost = result_pred_df['y_pred_v0'].values

In [ ]:
predictions_catboost.min(), predictions_catboost.max()

In [ ]:
normalized_predictions_catboost = (predictions_catboost - predictions_catboost.min())/(predictions_catboost - predictions_catboost.min()).max()
normalized_predictions_catboost

In [ ]:
normalized_preds = (normalized_predictions_catboost + test_normalized_preds_lightfm) / 2

# Submission

In [ ]:
submission['rating'] = normalized_preds

In [ ]:
submission

In [ ]:
VERSION = 45

In [ ]:
submission.to_csv(f'submission_log_v{VERSION}.csv', index=False)

# NN_collab_filter

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
from matplotlib.pyplot import figure

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8)
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve, accuracy_score, f1_score, precision_score, recall_score

import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Embedding, Flatten, Dense, Dropout, concatenate, multiply, Input, BatchNormalization
from keras.optimizers import Adam
from keras import metrics
from keras.utils.vis_utils import plot_model

import scikitplot as skplt

import sys
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
    
RANDOM_SEED = 13

print('Python       :', sys.version.split('\n')[0])
print('Pandas       :', pd.__version__)
print('Numpy        :', np.__version__)
print('Keras        :', keras.__version__)

In [ ]:
def open_data():
    """ open datasets"""
    global train, test, sample_submission # объявляем переменные глобальными
    train = pd.read_csv('/kaggle/input/recommendationsv4/train.csv', low_memory = False)
    train = train.drop_duplicates().reset_index(drop = True) # удалим дубликаты, если есть
    test = pd.read_csv('/kaggle/input/recommendationsv4/test.csv', low_memory = False)
    sample_submission = pd.read_csv('/kaggle/input/recommendationsv4/sample_submission.csv')
    
open_data() # открываем все и записываем датасет в переменные

def param_data(data): # посмотрим на данные
    """dataset required parameters """
    param = pd.DataFrame({
              'dtypes': data.dtypes.values,
              'nunique': data.nunique().values,
              'isna': data.isna().sum().values,
              'loc[0]': data.loc[0].values,
              }, 
             index = data.loc[0].index)
    return param

pd.concat([param_data(train), param_data(test)], 
          axis=1, 
          keys = [f'↓ ОБУЧАЮЩАЯ ВЫБОРКА ↓ {train.shape}', f'↓ ТЕСТОВАЯ ВЫБОРКА ↓ {test.shape}'],  
          sort=False)

In [ ]:
def viz_na(data):
    """NA visualisation"""
    global cols
    cols = data.columns # запишем названия строки сделаем переменную глобальной
    # определяем цвета 
    # желтый - пропущенные данные, синий - не пропущенные
    colours = ['#000099', '#ffff00'] 
    sns.heatmap(data[cols].isnull(), cmap=sns.color_palette(colours))
    plt.show()


viz_na(train)
viz_na(test)

In [ ]:
def stat_na_per_percent(data):
    print(f'{data.shape}')
    for col in data.columns:
        pct_missing = np.mean(data[col].isnull())
        print('{} - {}%'.format(col, round(pct_missing*100)))
    print("END", end = '\n\n')
stat_na_per_percent(train)
stat_na_per_percent(test)

In [ ]:
sns.countplot(train['rating']);

In [ ]:
sns.heatmap(train.corr())
plt.show()

In [ ]:
train_data_GB = pd.DataFrame({
    'userid': train['userid'],
    'itemid': train['itemid'],
    'rating': train['rating']
})

test_data_GB = pd.DataFrame({
    'userid': test['userid'],
    'itemid': test['itemid'],
})

train_data_GB = pd.get_dummies(train_data_GB, prefix='', prefix_sep='', columns=['rating'])

In [ ]:
#train_data_GB['rating'] = train_data_GB['rating'].astype(int)
## unisue users, books
userid, utemid = len(train_data_GB.userid.unique()), len(train_data_GB.itemid.unique())


f'The dataset includes {len(train_data_GB)} ratings by {userid} unique users on {utemid} unique itemid.'

In [ ]:
train_, test_ = train_test_split(train_data_GB, test_size=0.01)
f"The training and testing data include {len(train), len(test)} records."

### Tabular data method

In [ ]:
keras.backend.clear_session()
## define the number of latent factors (can be different for the users and books)
dim_embedding_user = 40
dim_embedding_book = 40

## book embedding
utem_input= Input(shape=[1], name='Utem')
utem_embedding = Embedding(utemid + 1, dim_embedding_book, name='Utem-Embedding')(utem_input)
utem_vec = Flatten(name='Utem-Flatten')(utem_embedding)
utem_vec = Dropout(0.5)(utem_vec)

## user embedding
user_input = Input(shape=[1], name='User')
user_embedding = Embedding(userid + 1, dim_embedding_user, name ='User-Embedding')(user_input)
user_vec = Flatten(name ='User-Flatten')(user_embedding)
user_vec = Dropout(0.5)(user_vec)

## concatenate flattened values 
concat = concatenate([utem_vec, user_vec])
concat_dropout = Dropout(0.5)(concat)

## add dense layer (can try more)
dense_1 = Dense(30, name ='Fully-Connected1', activation='relu')(concat)
dense_1 = Dense(30, name ='Fully-Connected2', activation='relu')(dense_1)
dense_1 = Dense(30, name ='Fully-Connected3', activation='relu')(dense_1)
dense_1 = Dense(30, name ='Fully-Connected4', activation='relu')(dense_1)
## define output (can try sigmoid instead of relu)
result = Dense(2, activation ='softmax',name ='Activation')(dense_1)

## define model with 2 inputs and 1 output
model_tabular = Model([user_input, utem_input], result)
model_tabular.summary()

In [ ]:
plot_model(model_tabular, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
## specify learning rate (or use the default by specifying optimizer = 'adam')
opt_adam = Adam(lr = 0.0003)

## compile model
model_tabular.compile(optimizer= opt_adam, loss= ['binary_crossentropy'], metrics=[keras.metrics.AUC()])

## fit model
history_tabular = model_tabular.fit([train_['userid'], train_['itemid']],
                                    train_.iloc[:,[-2,-1]],
                                    batch_size = 1000,
                                    validation_split = 0.1,
                                    epochs = 4,
                                    verbose = 1)

In [ ]:
test_pred_GB = model_tabular.predict([test_data_GB['userid'], test_data_GB['itemid']])[:,1]

In [ ]:
test_pred = model_tabular.predict([test_['userid'], test_['itemid']])

In [ ]:
skplt.metrics.plot_roc(test_.iloc[:, -1], test_pred)
plt.show()

In [ ]:
fpr, tpr, _ = roc_curve(test_.iloc[:, -1], test_pred[:,1])
auc = roc_auc_score(test_.iloc[:,  -1], test_pred[:, 1])
accuracy = accuracy_score(test_.iloc[:, -1], np.argmax(test_pred, axis = -1))
f1 = f1_score(test_.iloc[:, -1], np.argmax(test_pred, axis = -1))
precision =  precision_score(test_.iloc[:, -1], np.argmax(test_pred, axis = -1))
recall = recall_score(test_.iloc[:, -1], np.argmax(test_pred, axis = -1))

print('FP, TP              :', sum(fpr), sum(tpr))
print('ROC_AUC_SCORE       :', auc)
print('accuracy            :', accuracy)
print('precision           :', precision)
print('recall              :', recall)
print('f1                  :', f1)

In [ ]:
sample_submission['rating'] = test_pred_GB
sample_submission.to_csv('submission_XXXXXX.csv', index=False)
sample_submission.head(3)

In [ ]:
#Эмбединги
utem_embedding_weights = model_tabular.layers[2].get_weights()[0]

In [ ]:
utem_embedding_weights